In [1]:
import os
import cv2
import shutil

#### Convert a list of video folders to frames folders

In [7]:
def vid_to_frames(vid_path, output_dir, fps=25):

    # create output dir if it doesn't exist
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    # capture the video
    vid = cv2.VideoCapture(vid_path)

    # get original fps
    original_fps = vid.get(cv2.CAP_PROP_FPS)

    # calculate interval between frames
    interval = int(original_fps / fps)

    # initialize frame counters
    frame_count = 0 # counter for all frames
    saved_frame_count = 0 # counter for saved frames

    # loop through each frame of the video
    while True:
        ret, frame = vid.read() # read a frame from the video
        if not ret: # break the loop if no frame is returned
            break

        # check if the current frame should be saved based on the interval
        if frame_count % interval == 0:
            # create the filename for the saved frame
            frame_filename = os.path.join(output_dir, f"frame_{saved_frame_count:04d}.png")
            # save the frame as a png file
            cv2.imwrite(frame_filename, frame)
            saved_frame_count += 1
        
        frame_count += 1
    
    vid.release()

    print(f"Extracted {saved_frame_count} frames at {fps} FPS.")

#### Converts videos in a directory to frames in respective folders based on the video file name

In [8]:
# input_dir = ./data/vidtimit/deepfake/videos
# output_dir = ./data/vidtimit/deepfake/frames

def vid_folders_to_frame_folders(input_dir, output_dir, fps=25):
    
    for video in os.listdir(input_dir):

        # new folder name
        new_dir = os.path.join(output_dir,video)

        if not os.path.exists(new_dir):
            os.makedirs(new_dir)
        
        vid_to_frames(os.path.join(input_dir, video), new_dir)

In [ ]:
# vid_folders_to_frame_folders('./data/vidtimit/deepfake/videos', './data/vidtimit/deepfake/frames')

#### Convert Video to Frames

In [ ]:
def vid_to_frames(vid_path, output_dir, fps=25):

    # create output dir if it doesn't exist
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    # capture the video
    vid = cv2.VideoCapture(vid_path)

    # get original fps
    original_fps = vid.get(cv2.CAP_PROP_FPS)

    # calculate interval between frames
    interval = int(original_fps / fps)

    # initialize frame counters
    frame_count = 0 # counter for all frames
    saved_frame_count = 0 # counter for saved frames

    # loop through each frame of the video
    while True:
        ret, frame = vid.read() # read a frame from the video
        if not ret: # break the loop if no frame is returned
            break

        # check if the current frame should be saved based on the interval
        if frame_count % interval == 0:
            # create the filename for the saved frame
            frame_filename = os.path.join(output_dir, f"frame_{saved_frame_count:04d}.png")
            # save the frame as a png file
            cv2.imwrite(frame_filename, frame)
            saved_frame_count += 1
        
        frame_count += 1
    
    vid.release()

    print(f"Extracted {saved_frame_count} frames at {fps} FPS.")


#### Cool stuff

In [ ]:
# create a facelandmarker instance with the video mode:
options = FaceLandmarkerOptions(
    base_options = BaseOptions(model_asset_path='./models/face_landmarker.task'),
    running_mode=VisionRunningMode.VIDEO,
    output_face_blendshapes=True,
    output_facial_transformation_matrixes=True,
    num_faces=1
)

with FaceLandmarker.create_from_options(options) as landmarker:

    transformation_matrices = []
    timestamps = []

    #process video frames
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        # convert the frame to RGB as MediaPipe expects RGB format
        frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

        # create a MediaPipe Image Object
        mp_image = mp.Image(image_format=mp.ImageFormat.SRGB, data=frame_rgb)

        # get the frame timestamp
        frame_timestamp = int(cap.get(cv2.CAP_PROP_POS_MSEC))

        result = landmarker.detect_for_video(mp_image, frame_timestamp)


        # draw landmarks on the frame
        if result.face_landmarks:
            for matrix in result.face_landmarks:
                transformation_matrices.append(matrix)
                timestamps.append(frame_timestamp)
                for m in matrix:
                    x, y = int(m.x * frame.shape[1]), int(m.y * frame.shape[0])
                    cv2.circle(frame, (x,y), 2, (0,255,0), -1)

        cv2.imshow('Face Landmarker', frame)

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()

    cv2.destroyAllWindows()
